<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Package Import </span>

In [5]:
import string
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from bs4 import BeautifulSoup
from ipywidgets import Layout
from IPython.display import display, clear_output


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load Artists Library </span>

In [6]:
%run py_utils/generic_utils.ipynb

start_urls = []
path = '../datasets/artists_data.json'
all_artists = load_json(path)

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Artist Selection Interface </span>

In [7]:
%run py_utils/widgets_utils.ipynb

max_cache = 3
alphabets = [letter for letter in string.ascii_uppercase]
alphabets.append('#')
user_selection = []

def get_artists():
    def alphabet_based_artists(change):
        flag = 0
        artists = []
        artists_alt = []
        selected_alphabet = ''.join(change.new)         
        
        for i in all_artists['Artists']:
            if i[0].upper() == selected_alphabet:
                artists.append(i)
                flag = 1
                
            elif i[0].upper() not in alphabets or i[0] == '#':                
                artists_alt.append(i)
        
        if flag: artist_dropdown.options = artists; artist_dropdown.value = artists[0]
        else: artist_dropdown.options = artists_alt; artist_dropdown.value = artists_alt[0]
    
    def display_selected_artist(change):
        selection = ''.join(change.new)
        widget_output(output, selection)
        artist_dropdown.value = selection
            
    def retrieve_data(_): 
        if len(user_selection) < max_cache:
            comment = artist_dropdown.value+" saved."
            user_selection.append(artist_dropdown.value)
            widget_output(output, comment)
            
        else:
            comment = "Gold membership is required to select more than 3 artists."
            widget_output(output, comment)
    
    def clear_data(_):
        user_selection.clear()
        comment = "Selection Cleared."
        widget_output(output, comment)
    
    output = create_output()
    alphabet_dropdown = assign_dropdown(alphabets, "Alphabet", arg_function=alphabet_based_artists, observe=True)
    artist_dropdown = assign_dropdown(all_artists, "Artist", arg_function=display_selected_artist, observe=True)    
    dropdowns = create_gui(alphabet_dropdown, artist_dropdown, num=[0,1], wd='300', widget_type='H', gui_display=0)
    
    select_button = create_button("Select Artist", arg_function=retrieve_data)
    clear_button = create_button("Clear Selection", arg_function=clear_data)
    buttons = create_gui(select_button, clear_button, num=[0,1], wd='300', widget_type='H', gui_display=0)    
    create_gui(output, dropdowns, buttons, widget_type='V', gui_display=1)  
    

In [8]:
get_artists()

In [9]:
user_selection

['Coldplay']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Search dictionary and store relevant URLs </span>

In [10]:
%run py_utils/azlyrics_utils.ipynb

start_urls = match_artists(user_selection, all_artists)
start_urls

['https://www.azlyrics.com/c/coldplay.html']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Scrape tracks from artists </span>

In [11]:
%run crawlers/tracks_crawler.ipynb

tracks_crawler(start_urls)

> Json storage is successful.
> Dataframe storage is successful.
> Dataframe storage is successful.


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load discography </span>

In [12]:
json_path = '../datasets/discography_data.json'
discography = load_json(json_path)

discography.keys()

dict_keys(['Tracks', 'Artists', 'Albums', 'URLs'])

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Track Selection Interface </span>

In [18]:
%run py_utils/generic_utils.ipynb
%run py_utils/azlyrics_utils.ipynb

max_cache = 3
user_artist = []
user_album = []
user_track = []

def get_discography():    
    def artist_based_albums(change):
        selected_artists = ''.join(change.new)        
        albums_list = get_albums(discography, selected_artists)
        
        album_dropdown.options = albums_list
        album_dropdown.value = albums_list[0]
                
    def album_based_tracks(change):
        selected_albums = ''.join(change.new) 
        tracks_list = get_tracks(discography, selected_albums)
        
        track_dropdown.options = tracks_list
        track_dropdown.value = tracks_list[0]
    
    def display_selected_track(change):
        selection = change.new
        widget_output(output, selection)
        track_dropdown.value = selection        
        
    def retrieve_track(_):            
        if len(user_artist) < max_cache:
            comment = track_dropdown.value + " saved."
            selected_artist = ''.join(artist_dropdown.value)
            selected_album = ''.join(album_dropdown.value) if selected_artist else None
            selected_track = ''.join(track_dropdown.value) if selected_album else None
            
            user_artist.append(selected_artist)
            user_album.append(selected_album)
            user_track.append(selected_track)
            widget_output(output, comment)
        else:
            comment = "Note: Free Membership only allows up to 3 songs."
            widget_output(output, comment)            

    def reset_selection(_):
        user_artist.clear()
        user_album.clear()
        user_track.clear()
        comment = "Selection Cleared."
        widget_output(output, comment)  
        
    def refresh_selection(_):
        selected_artists = ''.join(artist_dropdown.value)
        albums_list = get_albums(discography, selected_artists) 
        
        album_dropdown.options = albums_list
        album_dropdown.value = albums_list[0] 
    
    output = create_output()    
    artist_dropdown = assign_dropdown(set(discography['Artists']), "Select Artists", arg_function=artist_based_albums, 
                                                observe=1)    
    album_dropdown = assign_dropdown([], "Select Album", arg_function=album_based_tracks, observe=1)    
    track_dropdown = assign_dropdown([], "Select Tracks", arg_function=display_selected_track, observe=1)
    button_track = create_button("Select Track", arg_function=retrieve_track)
    button_reset = create_button("Clear Selection", arg_function=reset_selection)    
    button_refresh = create_button("Refresh", arg_function=refresh_selection)
    
    dropdowns = create_gui(artist_dropdown, album_dropdown, track_dropdown, num=[0,1,2], wd='350', 
                           widget_type='H', gui_display=0)
    buttons = create_gui(button_track, button_reset, button_refresh, num=[0,1,2], wd='250', widget_type='H', gui_display=0)
    create_gui(dropdowns, buttons, output, widget_type='V', gui_display=1)
    

In [19]:
get_discography()

In [17]:
user_track

['We Never Change']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Generate URLs from the selected tracks </span>

In [37]:
%run py_utils/azlyrics_utils.ipynb
start_urls= []

for i, track in enumerate(user_track):
    start_urls.append(get_url(discography, track, user_artist[i], user_album[i]))

start_urls

['https://www.azlyrics.com/lyrics/blacksabbath/bornagain.html',
 'https://www.azlyrics.com/lyrics/beatles/slowdown.html',
 'https://www.azlyrics.com/lyrics/acdc/flickoftheswitch.html']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Extract lyrics data from tracks </span>

In [38]:
%run crawlers/lyrics_crawler.ipynb

lyrics_crawler(start_urls)

3 3 3 3 3
> Json storage is successful.
> Dataframe storage is successful.
> Dataframe storage is successful.


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load Lyrics </span>

In [39]:
%run py_utils/generic_utils.ipynb

json_path = '../datasets/lyrics_data.json'
track_lyrics = load_json(json_path)

track_lyrics.keys()

dict_keys(['Tracks', 'Lyrics', 'Artists', 'Albums', 'Credits'])

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Display Lyrics and Artist Information </span>

In [40]:
%run py_utils/azlyrics_utils.ipynb
%run py_utils/widgets_utils.ipynb
 
selection_size = len(track_lyrics['Tracks'])

def get_lyrics():        
    def select_track(change):        
        selected_track = ''.join(change.new)
        track_dropdown.value = selected_track
        widget_output(output, selected_track)
    
    def display_lyrics(_):
        selected_lyrics = ''.join(track_dropdown.value)        
        get_track_lyrics(output, track_lyrics, selected_lyrics)        

    output = create_output() 
    track_dropdown = assign_dropdown(set(track_lyrics['Tracks']), "Select Tracks", arg_function=select_track, observe=1)    
    lyrics_button = create_button("Display Lyrics", arg_function=display_lyrics)  
    
    dropdown_button = create_gui(track_dropdown, lyrics_button, widget_type='H', num=[0,1], wd='400', gui_display=0)
    create_gui( dropdown_button, output, widget_type='V', num=[1], wd='400', gui_display=1)    


In [41]:
get_lyrics()
